In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import pandas as pd
from shapely.geometry import Point
import warnings
from shapely.wkt import loads
import trackintel as ti
from tqdm import tqdm

In [3]:
stop_point_df = pd.read_csv('staypoints.csv')
stop_point_df

,id,user_id,started_at,finished_at,geom
0,0,1,2022-09-07 14:34:13-04:00,2022-09-07 14:39:10-04:00,POINT (-81.6264605000000074 30.1175764999999984)
1,1,1,2022-09-07 14:39:10-04:00,2022-09-07 14:44:28-04:00,POINT (-81.6267524999999949 30.1162075000000016)
2,2,1,2022-09-10 01:03:16-04:00,2022-09-10 01:11:18-04:00,POINT (-81.7973199999999991 30.1123399999999997)
3,3,1,2022-09-10 01:27:28-04:00,2022-09-10 01:35:20-04:00,POINT (-81.7621800000000007 30.3330200000000012)
4,4,1,2022-09-10 01:35:20-04:00,2022-09-10 01:39:20-04:00,POINT (-81.7086500000000058 30.4537799999999983)
...,...,...,...,...,...
242026,242026,12414,2022-10-31 21:23:39-04:00,2022-10-31 21:28:30-04:00,POINT (-81.6689142999999973 30.3620313449999983)
242027,242027,12415,2022-10-27 03:34:53-04:00,2022-10-27 03:37:40-04:00,POINT (-81.6483199999999982 30.3217199999999991)
242028,242028,12415,2022-10-27 05:30:52-04:00,2022-10-27 05:38:14-04:00,POINT (-81.6525199999999955 30.3216099999999997)
242029,242029,12415,2022-10-27 05:38:14-04:00,2022-10-27 05:53:02-04:00,POINT (-81.6508199999999960 30.3522199999999991)


In [4]:
retail_pd = pd.read_csv('retails_Jacksonville_FL.csv')
retail_pd

,Unnamed: 0,ObjectId,Store_Name,Address,Address_Li,City,State,Zip5,Zip4,County,Longitude,Latitude,geometry
0,1105,1106,The Pit Stop 2,996 Saint Johns Bluff Rd N,NaN,Jacksonville,FL,32225,8386.0,DUVAL,-81.523949,30.338488,POINT (-81.52394899999996 30.33848800000004)
1,1214,1215,James Food Mart,2651 Jammes Rd,NaN,Jacksonville,FL,32210,3751.0,DUVAL,-81.747391,30.271931,POINT (-81.74739099999994 30.271931000000052)
2,1856,1857,Vibe Smoke & Mini Mart,619 University Blvd N,NaN,Jacksonville,FL,32211,7037.0,DUVAL,-81.604263,30.326391,POINT (-81.60426299999995 30.326391000000058)
3,1882,1883,IFG Jacksonville 344,3100 Hilton St,NaN,Jacksonville,FL,32209,2750.0,DUVAL,-81.715515,30.365610,POINT (-81.71551499999998 30.36561000000006)
4,2323,2324,Jacksonville 76 Petroleum Llc,5971 Ramona Blvd,NaN,Jacksonville,FL,32205,4755.0,DUVAL,-81.751587,30.314375,POINT (-81.75158699999997 30.31437500000004)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,244425,244426,J & W Discount,4045 Post St,NaN,Jacksonville,FL,32205,5388.0,DUVAL,-81.719048,30.311987,POINT (-81.71904799999999 30.311987000000045)
845,244595,244596,Ajs Seafood Llc 2 2,725 Edgewood Ave N,NaN,Jacksonville,FL,32254,3013.0,DUVAL,-81.730835,30.331091,POINT (-81.73083499999996 30.33109100000007)
846,244629,244630,Abbey Food Store,6164 Cleveland Rd,NaN,Jacksonville,FL,32209,1923.0,DUVAL,-81.715858,30.377815,POINT (-81.71585799999997 30.377815000000055)
847,244891,244892,Soriya Food Market,3561 Emerson St,NaN,Jacksonville,FL,32207,5628.0,DUVAL,-81.628120,30.291224,POINT (-81.62811999999997 30.291224000000057)


In [5]:
def is_close(dist, latA1, lonA2, latB1, lonB2):
    # Create shapely Point objects from the locations
    point1 = Point(lonA2, latA1)
    point2 = Point(lonB2, latB1)
    
    # Calculate distance between points in meters
    distance = point1.distance(point2) * 111319.9
    
    # Return True if distance is less than or equal to dist meters
    return distance <= dist

In [6]:
count = 0
warnings.simplefilter(action='ignore', category=Warning)
data_to_append_list = []
for sp in tqdm(stop_point_df.itertuples(), total=len(stop_point_df)):
    p = loads(sp[5])
    lat = p.y
    lon = p.x
    
    for row in retail_pd.itertuples():
        if is_close(200, float(row[12]), float(row[11]), lat, lon):
            count += 1
            
            data_to_append = {'id': sp[1], 'user_id': sp[2], 'started_at': sp[3], 'finished_at': sp[4], 'lat': lat, 'lon': lon, 'retail_id': row[1], 
                                'retail_lat': float(row[12]), 'retail_lon': float(row[11])}
            data_to_append_list.append(data_to_append)
            
            break

new_df = pd.DataFrame.from_records(data_to_append_list)
print('Point close to retail: ', count)

100%|██████████| 242031/242031 [1:27:45<00:00, 45.97it/s] 

Point close to retail:  35445


In [7]:
new_df

,id,user_id,started_at,finished_at,lat,lon,retail_id,retail_lat,retail_lon
0,0,1,2022-09-07 14:34:13-04:00,2022-09-07 14:39:10-04:00,30.117576,-81.626461,36037,30.118788,-81.625732
1,10,1,2022-09-23 12:16:32-04:00,2022-09-23 12:26:14-04:00,30.167915,-81.631710,168227,30.167103,-81.631638
2,14,1,2022-09-24 20:52:52-04:00,2022-09-24 20:54:58-04:00,30.146970,-81.632480,151588,30.147409,-81.632477
3,23,1,2022-09-30 00:42:04-04:00,2022-09-30 00:51:33-04:00,30.269802,-81.773450,163914,30.269903,-81.772171
4,27,1,2022-10-09 12:55:19-04:00,2022-10-09 13:07:11-04:00,30.117490,-81.626550,36037,30.118788,-81.625732
...,...,...,...,...,...,...,...,...,...
35440,241994,12388,2022-10-27 19:34:13-04:00,2022-10-27 19:37:51-04:00,30.268400,-81.613899,237958,30.268553,-81.613953
35441,242001,12388,2022-10-31 19:12:20-04:00,2022-10-31 19:14:39-04:00,30.269813,-81.610858,45499,30.270267,-81.610809
35442,242008,12399,2022-10-29 05:57:54-04:00,2022-10-29 06:02:57-04:00,30.196416,-81.744182,119875,30.196510,-81.744141
35443,242022,12411,2022-10-26 23:04:35-04:00,2022-10-26 23:07:15-04:00,30.322587,-81.557411,58187,30.321955,-81.556618


In [8]:
print(len(new_df['user_id'].unique()))

2517


In [9]:
new_df.to_csv('food_inference.csv')